
# GenAI observability with MLflow Tracing

It is highly recommended that you enable [MLflow Tracing](https://docs.databricks.com/aws/en/mlflow3/genai/tracing) when developing GenAI applications.

Tracing provides a step-by-step analysis of your app's execution, helping you debug latency, cost, and quality issues.

Traces can be used with [Agent Evaluation](https://docs.databricks.com/aws/en/generative-ai/agent-evaluation/evaluate-agent) to measure your app's quality, and the same Trace will be [automatically captured](https://docs.databricks.com/aws/en/generative-ai/agent-framework/deploy-agent#agent-enhanced-inference-tables) once you deploy your application.

MLflow tracing can be enabled as follows:
<br><br>
```python
import mlflow

mlflow.dspy.autolog()
```


## 📚 Articles

1. [Native OpenAI Models Now Generally Available on Databricks](https://www.databricks.com/blog/native-openai-models-now-generally-available-databricks)
1. [Launching Gemini 3 Pro on Databricks](https://www.databricks.com/blog/launching-gemini-3-pro-databricks)
1. [Building Custom LLM Judges for AI Agent Accuracy](https://www.databricks.com/blog/building-custom-llm-judges-ai-agent-accuracy)
1. [Accelerate AI Development with Databricks: Discover, Govern, and Build with MCP and Agent Bricks](https://www.databricks.com/blog/accelerate-ai-development-databricks-discover-govern-and-build-mcp-and-agent-bricks)
1. [PDFs to Production: Announcing state-of-the-art document intelligence on Databricks](https://www.databricks.com/blog/pdfs-production-announcing-state-art-document-intelligence-databricks)
1. [Configure environment for job tasks](https://docs.databricks.com/aws/en/compute/serverless/dependencies#libraries)
1. [Manage model lifecycle in Unity Catalog](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/manage-model-lifecycle/)
1. [Monitor GenAI in production](https://docs.databricks.com/aws/en/mlflow3/genai/eval-monitor/production-monitoring)
1. [Agent Bricks](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-bricks/)
1. [Accelerate AI Development with Databricks: Discover, Govern, and Build with MCP and Agent Bricks](https://www.databricks.com/blog/accelerate-ai-development-databricks-discover-govern-and-build-mcp-and-agent-bricks)
1. [Create and edit prompts](https://learn.microsoft.com/en-us/azure/databricks/mlflow3/genai/prompt-version-mgmt/prompt-registry/create-and-edit-prompts)
1. [Building Custom LLM Judges for AI Agent Accuracy](https://www.databricks.com/blog/building-custom-llm-judges-ai-agent-accuracy)

In [0]:
%pip install -U dspy==3.0.4 mlflow==3.7.0
dbutils.library.restartPython()


Interesting transitive dependencies are installed alongside (100% Python 🐍):

1. [colorlog](https://github.com/borntyping/python-colorlog)
1. [optuna](https://github.com/optuna/optuna)

In [0]:
import mlflow

mlflow.get_tracking_uri()

In [0]:
import mlflow

print(f"MLflow version: {mlflow.__version__}")

## List available model (serving) endpoints

In [0]:
from mlflow.deployments import get_deploy_client
from pyspark.sql.types import StructType, StructField, StringType, MapType

deploy_client = get_deploy_client("databricks")
endpoints = deploy_client.list_endpoints()

endpoint_names = sorted([endpoint['name'] for endpoint in endpoints])

# Wrap each name in a tuple to match the schema to build a DataFrame from
endpoint_names_df = spark.createDataFrame(
    [(name,) for name in endpoint_names],
    schema=StructType([StructField("Endpoint Name", StringType())])
)

display(endpoint_names_df)

In [0]:
from openai import OpenAI

databricks_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
databricks_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    base_url=f"{databricks_url}/serving-endpoints",
    api_key=databricks_token,
)

response = client.chat.completions.create(
    model="databricks-claude-sonnet-4-5",
    messages=[
        {
            "role": "user",
            "content": "What is an LLM agent?"
        }
    ],
    max_tokens=5000
)

print(response.choices[0].message.content)


## LLM Agent

An **LLM agent** is an AI system that uses a Large Language Model (LLM) as its "brain" to autonomously perform tasks, make decisions, and interact with tools or environments to achieve specific goals.

## Key Components

**Core LLM**: The reasoning engine (like GPT-4, Claude, etc.) that processes information and decides actions

**Agency**: Unlike a basic chatbot, an agent can:
- Plan multi-step tasks
- Make decisions autonomously
- Take actions in sequence
- Adapt based on results

**Tools/Actions**: Agents can interact with external systems:
- Search the web
- Execute code
- Query databases
- Call APIs
- Use calculators

**Memory**: Track conversation history and intermediate results

## How It Works

1. **Receive a goal** (e.g., "Book me a flight to Paris")
2. **Plan** steps needed to accomplish it
3. **Act** by using available tools
4. **Observe** the results
5. **Reason** about next steps
6. **Repeat** until goal is achieved

## Example Use Cases

- Research assistants that gather and synthesize information
- Customer service bots that can check orders and process refunds
- Coding assistants that can write, test, and debug code
- Personal assistants that manage calendars and emails

The key difference from a standard LLM is the **action loop** - agents don't just respond once, they actively work toward completing objectives.#

# DSPy

In [0]:
import dspy

print(f"DSPy version: {dspy.__version__}")

In [0]:
import dspy

lm = dspy.LM(model="databricks/databricks-claude-sonnet-4-5")
dspy.configure(lm=lm)

In [0]:
lm('hello')

## Enable MLflow DSPy tracing (autologging)

In [0]:
import mlflow

mlflow.dspy.autolog()

In [0]:
help(mlflow.dspy.autolog)

In [0]:
lm('hello')


## dspy.Predict

In [0]:
# Direct copy of the opening example at https://dspy.ai/learn/programming/signatures/
my_toxic_signature = dspy.Signature(
        "comment -> toxic: bool",
        instructions="Mark as 'toxic' if the comment includes insults, harassment, or sarcastic derogatory remarks.",
    )
toxicity = dspy.Predict(my_toxic_signature)
comment = "you are beautiful."
toxicity(comment=comment).toxic